In [2]:

!pip install openpyxl
!pip install xlsxwriter
import pandas as pd
import numpy as np
import io
import os
from openpyxl import Workbook
from openpyxl.styles import PatternFill
from openpyxl.utils.dataframe import dataframe_to_rows
from google.colab import files
uploaded = files.upload()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.4/169.4 kB 2.9 MB/s eta 0:00:00


Saving placement.csv to placement.csv


In [11]:

filename =list(uploaded.keys())[0]
file_exten=os.path.splitext(filename)[1].lower()

if file_exten=='.xlsx':
  df=pd.read_excel(io.BytesIO(uploaded[filename]),engine='openpyxl')
elif file_exten=='.csv':
  df=pd.read_csv(io.BytesIO(uploaded[filename]))
elif file_exten=='.xls':
  df=pd.read_excel(io.BytesIO(uploaded[filename]),engine='xlrd')
else:
  print('Unsupported file format')

def autclean(df):
    dfc = df.copy()

    total_rows = len(dfc)

    for col in dfc.columns:
        null_count = dfc[col].isnull().sum()

        if null_count >= (total_rows / 4):
            # Fill missing values
            if pd.api.types.is_numeric_dtype(dfc[col]):
                mean_val = dfc[col].mean()
                print(f"Filling numeric column '{col}' with mean: {mean_val}")
                dfc[col] = dfc[col].fillna(mean_val)
            else:
                print(f"Filling non-numeric column '{col}' with 'Missing'")
                dfc[col] = dfc[col].fillna("Missing")
        else:
            before_drop = len(dfc)
            dfc = dfc[dfc[col].notnull()]
            after_drop = len(dfc)
            print(f"Dropped {before_drop - after_drop} rows due to missing in column '{col}'")

    return dfc

dfcl = autclean(df)

print("\nMissing values after cleaning:")
print(dfcl.isnull().sum())

def autstat(df):
    numeric_df = df.select_dtypes(include=np.number)

    stats = {
        'mean': numeric_df.mean(),
        'median': numeric_df.median(),
        'mode': numeric_df.mode().iloc[0] if not numeric_df.mode().empty else pd.Series(index=numeric_df.columns),
        'z_scores': (numeric_df - numeric_df.mean()) / numeric_df.std()
    }

    stats['outliers'] = (np.abs(stats['z_scores']) > 2)
    return stats

def get_fill(color_hex):
    return PatternFill(start_color=color_hex, end_color=color_hex, fill_type="solid")

HIGHLIGHT = {
    'outlier': get_fill("FF6666"),
    'mean': get_fill("99CCFF"),
    'median': get_fill("99FF99"),
    'mode': get_fill("FFFF99"),
}

def goodsheet(df,stats):
  wb=Workbook()
  ws=wb.active
  ws.title="Cleaned Data"
  ws.append(df.columns.tolist())
  for _, row in df.iterrows():
    row_values=row.tolist()
    ws.append(row.tolist())

  for col_idx, col in enumerate(df.select_dtypes(include=np.number).columns, 1):
        for row_idx in range(len(df)):
            cell = ws.cell(row=row_idx + 2, column=col_idx)
            val = df[col].iloc[row_idx]
            if stats['outliers'].loc[row_idx, col]:
                cell.fill = HIGHLIGHT['outlier']
            elif val == stats['mean'][col]:
                cell.fill = HIGHLIGHT['mean']
            elif val == stats['median'][col]:
                cell.fill = HIGHLIGHT['median']
            elif val == stats['mode'][col]:
                cell.fill = HIGHLIGHT['mode']
  return wb

def add_summary_sheet(wb, df, stats):
    ws = wb.create_sheet(title="Summary & Patterns")
    ws.append(["Column", "Mean", "Median", "Mode"])
    for col in df.select_dtypes(include=np.number).columns:
        ws.append([col, stats['mean'][col], stats['median'][col], stats['mode'][col]])

    ws.append([])
    ws.append(["Correlation Matrix"])
    corr = df.select_dtypes(include=np.number).corr()
    ws.append([""] + list(corr.columns))
    for idx, row in corr.iterrows():
        ws.append([idx] + row.tolist())


def export_excel(wb, output_path="/content/cleaned_output.xlsx"):
    wb.save(output_path)
    files.download(output_path)

stats = autstat(dfcl)
wb = goodsheet(dfcl, stats)
add_summary_sheet(wb, dfcl, stats)
export_excel(wb)




Dropped 0 rows due to missing in column 'sl_no'
Dropped 0 rows due to missing in column 'gender'
Dropped 0 rows due to missing in column 'ssc_p'
Dropped 0 rows due to missing in column 'ssc_b'
Dropped 0 rows due to missing in column 'hsc_p'
Dropped 0 rows due to missing in column 'hsc_b'
Dropped 0 rows due to missing in column 'hsc_s'
Dropped 0 rows due to missing in column 'degree_p'
Dropped 0 rows due to missing in column 'degree_t'
Dropped 0 rows due to missing in column 'workex'
Dropped 0 rows due to missing in column 'etest_p'
Dropped 0 rows due to missing in column 'specialisation'
Dropped 0 rows due to missing in column 'mba_p'
Dropped 0 rows due to missing in column 'status'
Filling numeric column 'salary' with mean: 288655.4054054054

Missing values after cleaning:
sl_no             0
gender            0
ssc_p             0
ssc_b             0
hsc_p             0
hsc_b             0
hsc_s             0
degree_p          0
degree_t          0
workex            0
etest_p        

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>